In [ ]:
import math
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import warnings

warnings.filterwarnings(action='ignore')
torch.set_printoptions(sci_mode=True)

DATA_PATH = '/opt/ml/final-project-level3-recsys-05/Model/Model-Experiment/data'
MODEL_PATH = '/opt/ml/final-project-level3-recsys-05/Model/Model-Experiment/model'
VAL_TO_IDX_DATA_PATH = '/opt/ml/final-project-level3-recsys-05/Model/Model-Server/data'

In [ ]:
import gc

gc.collect()
torch.cuda.empty_cache()

In [ ]:
def seed_everything(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)

## 데이터셋 구축

In [ ]:
df = pd.read_csv(os.path.join(DATA_PATH, 'score.csv'))
df = df[~(df['problem_id'].isna())].reset_index(drop = True)
df['problem_id'] = df['problem_id'].astype(int).astype(str)
df['target'] = df['target'].astype(int).astype(str)
df['key'] = df['user_name'] + '-' + df['target']

new_df = []

group_df = df.groupby('key')

for key, g_df in tqdm(group_df):
    new_df.append(g_df[~(g_df.duplicated('problem_id'))].copy())

new_df = pd.concat(new_df).reset_index(drop = True)

100%|██████████| 213030/213030 [04:04<00:00, 872.28it/s]


In [ ]:
import json

with open(os.path.join(VAL_TO_IDX_DATA_PATH, 'problemId_to_idx.json'), 'r', encoding = 'utf-8') as f:
    problemId_to_idx = json.load(f)

with open(os.path.join(VAL_TO_IDX_DATA_PATH, 'idx_to_problemId.json'), 'r', encoding = 'utf-8') as f:
    idx_to_problemId = json.load(f)

In [ ]:
def get_problemId2idx(x):
    if x in problemId_to_idx: return problemId_to_idx[x] + 1
    else: return np.nan
    
def get_target2idx(x):
    if x in problemId_to_idx: return problemId_to_idx[x]
    else: return np.nan
    
new_df['problemId2idx'] = new_df['problem_id'].apply(lambda x : get_problemId2idx(x))
new_df = new_df[~(new_df['problemId2idx'].isna())].reset_index(drop = True)
new_df['problemId2idx'] = new_df['problemId2idx'].astype(int)

new_df['target2idx'] = new_df['target'].apply(lambda x : get_target2idx(x))
new_df = new_df[~(new_df['target2idx'].isna())].reset_index(drop = True)
new_df['target2idx'] = new_df['target2idx'].astype(int)

## 데이터셋 분리

In [ ]:
train_df = []
valid_df = []

group_df = new_df.groupby('target')
for target, g_df in tqdm(group_df):
    gg_df = g_df.groupby('key')
    for idx, (userID, ggg_df) in enumerate(gg_df):
        if idx == 0:
            valid_df.append(ggg_df)
        else:
            train_df.append(ggg_df)

100%|██████████| 19764/19764 [00:18<00:00, 1074.66it/s]


In [ ]:
train_df = pd.concat(train_df).reset_index(drop = True)
valid_df = pd.concat(valid_df).reset_index(drop = True)

In [ ]:
train_df.to_csv(os.path.join(DATA_PATH, 'train_df.csv'), index = False)
valid_df.to_csv(os.path.join(DATA_PATH, 'valid_df.csv'), index = False)

## 모델

In [ ]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self, hidden_units, dropout_rate):
        super(ScaledDotProductAttention, self).__init__()
        self.hidden_units = hidden_units
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, Q, K, V, mask):
        """
        Q, K, V : (batch_size, num_heads, max_len, hidden_units)
        mask : (batch_size, 1, max_len, max_len)
        """
        attn_score = torch.matmul(Q, K.transpose(2, 3)) / math.sqrt(self.hidden_units) # (batch_size, num_heads, max_len, max_len)
        attn_score = attn_score.masked_fill(mask == 0, -1e9)  # 유사도가 0인 지점은 -infinity로 보내 softmax 결과가 0이 되도록 함
        attn_dist = self.dropout(F.softmax(attn_score, dim=-1))  # attention distribution
        output = torch.matmul(attn_dist, V)  # (batch_size, num_heads, max_len, hidden_units) / # dim of output : batchSize x num_head x seqLen x hidden_units
        return output, attn_dist


class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, hidden_units, dropout_rate):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads # head의 수
        self.hidden_units = hidden_units
        
        # query, key, value, output 생성을 위해 Linear 모델 생성
        self.W_Q = nn.Linear(hidden_units, hidden_units, bias=False)
        self.W_K = nn.Linear(hidden_units, hidden_units, bias=False)
        self.W_V = nn.Linear(hidden_units, hidden_units, bias=False)
        self.W_O = nn.Linear(hidden_units, hidden_units, bias=False)

        self.attention = ScaledDotProductAttention(hidden_units, dropout_rate)
        self.dropout = nn.Dropout(dropout_rate) # dropout rate
        self.layerNorm = nn.LayerNorm(hidden_units, 1e-6) # layer normalization

    def forward(self, enc, mask):
        """
        enc : (batch_size, max_len, hidden_units)
        mask : (batch_size, 1, max_len, max_len)
        
        """
        residual = enc # residual connection을 위해 residual 부분을 저장
        batch_size, seqlen = enc.size(0), enc.size(1)

        # Query, Key, Value를 (num_head)개의 Head로 나누어 각기 다른 Linear projection을 통과시킴
        Q = self.W_Q(enc).view(batch_size, seqlen, self.num_heads, self.hidden_units // self.num_heads) # (batch_size, max_len, num_heads, hidden_units)
        K = self.W_K(enc).view(batch_size, seqlen, self.num_heads, self.hidden_units // self.num_heads) # (batch_size, max_len, num_heads, hidden_units)
        V = self.W_V(enc).view(batch_size, seqlen, self.num_heads, self.hidden_units // self.num_heads) # (batch_size, max_len, num_heads, hidden_units)

        # Head별로 각기 다른 attention이 가능하도록 Transpose 후 각각 attention에 통과시킴
        Q, K, V = Q.transpose(1, 2), K.transpose(1, 2), V.transpose(1, 2) # (batch_size, num_heads, max_len, hidden_units)
        output, attn_dist = self.attention(Q, K, V, mask) # output : (batch_size, num_heads, max_len, hidden_units) / attn_dist : (batch_size, num_heads, max_len, max_len)

        # 다시 Transpose한 후 모든 head들의 attention 결과를 합칩니다.
        output = output.transpose(1, 2).contiguous() # (batch_size, max_len, num_heads, hidden_units) / contiguous() : 가변적 메모리 할당
        output = output.view(batch_size, seqlen, -1) # (batch_size, max_len, hidden_units * num_heads)

        # Linear Projection, Dropout, Residual sum, and Layer Normalization
        output = self.layerNorm(self.dropout(self.W_O(output)) + residual) # (batch_size, max_len, hidden_units)
        return output, attn_dist


class PositionwiseFeedForward(nn.Module):
    def __init__(self, hidden_units, dropout_rate):
        super(PositionwiseFeedForward, self).__init__()

        self.W_1 = nn.Linear(hidden_units, hidden_units)
        self.W_2 = nn.Linear(hidden_units, hidden_units)
        self.dropout = nn.Dropout(dropout_rate)
        self.layerNorm = nn.LayerNorm(hidden_units, 1e-6) # layer normalization

    def forward(self, x):
        residual = x
        output = self.W_2(F.relu(self.dropout(self.W_1(x))))
        output = self.layerNorm(self.dropout(output) + residual)
        return output


class SASRecBlock(nn.Module):
    def __init__(self, num_heads, hidden_units, dropout_rate):
        super(SASRecBlock, self).__init__()
        self.attention = MultiHeadAttention(num_heads, hidden_units, dropout_rate)
        self.pointwise_feedforward = PositionwiseFeedForward(hidden_units, dropout_rate)

    def forward(self, input_enc, mask):
        """
        input_enc : (batch_size, max_len, hidden_units)
        mask : (batch_size, 1, max_len, max_len)
        """
        output_enc, attn_dist = self.attention(input_enc, mask)
        output_enc = self.pointwise_feedforward(output_enc)
        return output_enc, attn_dist


class SASRec(nn.Module):
    def __init__(
        self, 
        num_assessmentItemID,
        hidden_units,
        num_heads, 
        num_layers, 
        dropout_rate):
        super(SASRec, self).__init__()

        self.assessmentItemID_emb = nn.Embedding(num_assessmentItemID + 1, hidden_units, padding_idx = 0) # 문항에 대한 정보

        self.blocks = nn.ModuleList([SASRecBlock(num_heads, hidden_units, dropout_rate) for _ in range(num_layers)])

        self.lstm = nn.LSTM(
            input_size = hidden_units,
            hidden_size = hidden_units,
            num_layers = num_layers,
            batch_first = True,
            bidirectional = False,
            dropout = dropout_rate,
            )
        
        self.predict_layer = nn.Sequential(
            nn.Linear(hidden_units, num_assessmentItemID)
        )
        
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    
    def forward(self, input):
        """
        assessmentItem : (batch_size, max_len)
        """
        assessmentItem = input['assessmentItem']

        # masking
        mask_pad = torch.BoolTensor(assessmentItem > 0).unsqueeze(1).unsqueeze(1) # (batch_size, 1, 1, max_len)
        mask_time = (1 - torch.triu(torch.ones((1, 1, assessmentItem.size(1), assessmentItem.size(1))), diagonal=1)).bool() # (batch_size, 1, max_len, max_len)
        mask = (mask_pad & mask_time).to(self.device) # (batch_size, 1, max_len, max_len)

        assessmentItem_emb = self.assessmentItemID_emb(assessmentItem.to(self.device))
        for block in self.blocks:
            assessmentItem_emb, attn_dist = block(assessmentItem_emb, mask)

        assessmentItem_emb, _ = self.lstm(assessmentItem_emb)
        
        output = self.predict_layer(assessmentItem_emb[:, -1])

        return output

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, df):

        self.features = []
        self.targets = []

        group_df = df.groupby('target')

        for target2idx, g_df in group_df:
            gg_df = g_df.groupby('key')
            for userID, ggg_df in gg_df:
                if len(ggg_df) >= 2:
                    self.features.append(ggg_df['problemId2idx'].tolist()[::-1][:-1])
                    self.targets.append(ggg_df['problemId2idx'].tolist()[::-1][-1] - 1)

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        feature = self.features[idx]
        target = self.targets[idx]
        return {
            'assessmentItem' : np.array(feature), 
            'target' : target,
            }


def pad_sequence(seq, max_len, padding_value = 0):
    try:
        seq_len, col = seq.shape
        padding = np.zeros((max_len - seq_len, col)) + padding_value
    except:
        seq_len = seq.shape[0]
        padding = np.zeros((max_len - seq_len, )) + padding_value

    padding_seq = np.concatenate([padding, seq])

    return padding_seq


def make_collate_fn(samples):
    max_len = 0
    for sample in samples:
        seq_len = sample['assessmentItem'].shape[0]
        if max_len < seq_len:
            max_len = seq_len
    
    features = []
    targets = []

    for sample in samples:
        features += [pad_sequence(sample['assessmentItem'], max_len = max_len, padding_value = 0)]
        targets += [sample['target']]

    return {
        'assessmentItem' : torch.tensor(features, dtype = torch.long), 
        'target': torch.tensor(targets, dtype = torch.long),
    }

In [ ]:
def get_ndcg(pred_list, true_list):
    idcg = sum((1 / np.log2(rank + 2) for rank in range(len(true_list))))
    dcg = 0
    for rank, pred in enumerate(pred_list):
        if pred in true_list:
            dcg += 1 / np.log2(rank + 2)
    ndcg = dcg / idcg
    return ndcg

def get_hit(pred_list, true_list):
    hit_list = set(true_list) & set(pred_list)
    hit = len(hit_list) / len(true_list)
    return hit

def get_rank(pred_list, true_list):
    ret_rank = len(pred_list) + 1
    for rank, pred in enumerate(pred_list):
        if pred in true_list:
            ret_rank = rank + 1
    return 0 if ret_rank == (len(pred_list) + 1) else ret_rank

def train(model, data_loader, criterion, optimizer):
    model.train()
    loss_val = 0

    for input in data_loader:

        optimizer.zero_grad()

        output = model(input)
        loss = criterion(output, input['target'].to(device))

        loss.backward()
        optimizer.step()

        loss_val += loss.item()

    loss_val /= len(data_loader)

    return loss_val

def evaluate(model, data_loader, topk = 10):
    model.eval()

    hit = 0
    ndcg = 0
    rank = 0

    with torch.no_grad():
        for input in data_loader:
            
            output = -model(input)
            predictions = output.argsort()

            targets = input['target']

            for target, prediction in zip(targets, predictions):
                target = [target.item()]
                prediction = prediction.cpu().tolist()[:topk]

                hit += get_hit(prediction, target)
                ndcg += get_ndcg(prediction, target)
                rank += get_rank(prediction, target)
    
    rank = rank / hit
    hit = hit / len(data_loader.dataset)
    ndcg = ndcg / len(data_loader.dataset)

    return hit, ndcg, rank

## 학습

In [ ]:
import json

with open(os.path.join(VAL_TO_IDX_DATA_PATH, 'problemId_to_idx.json'), 'r', encoding = 'utf-8') as f:
    problemId_to_idx = json.load(f)

with open(os.path.join(VAL_TO_IDX_DATA_PATH, 'idx_to_problemId.json'), 'r', encoding = 'utf-8') as f:
    idx_to_problemId = json.load(f)

In [ ]:
batch_size = 256
epochs = 20
lr = 0.001
device = 'cuda' if torch.cuda.is_available() else 'cpu'

hidden_units = 128
num_heads = 8
num_layers = 1
dropout_rate = 0.5
num_workers = 8
num_assessmentItemID = len(problemId_to_idx)

model_name = 'new-User-Seq-Transformer'

In [ ]:
train_df = pd.read_csv(os.path.join(DATA_PATH, 'train_df.csv'))
valid_df = pd.read_csv(os.path.join(DATA_PATH, 'valid_df.csv'))

In [ ]:
seed_everything(22)

In [ ]:
train_dataset = CustomDataset(df = train_df)
train_data_loader = DataLoader(
        train_dataset, 
        batch_size = batch_size, 
        shuffle = True, 
        drop_last = False,
        collate_fn = make_collate_fn,
        num_workers = num_workers)


valid_dataset = CustomDataset(df = valid_df)
valid_data_loader = DataLoader(
        valid_dataset, 
        batch_size = batch_size, 
        shuffle = False, 
        drop_last = False,
        collate_fn = make_collate_fn,
        num_workers = num_workers)

In [ ]:
model = SASRec(
        num_assessmentItemID = num_assessmentItemID,
        hidden_units = hidden_units,
        num_heads = num_heads,
        num_layers = num_layers,
        dropout_rate = dropout_rate
        ).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr = lr)
criterion = nn.CrossEntropyLoss()

In [ ]:
best_ndcg = 0

for epoch in range(1, epochs + 1):
    tbar = tqdm(range(1))
    for _ in tbar:
            train_loss = train(model = model, data_loader = train_data_loader, criterion = criterion, optimizer = optimizer)
            hit, ndcg, rank = evaluate(model = model, data_loader = valid_data_loader)
            if best_ndcg < ndcg:
                best_epoch = epoch
                best_train_loss = train_loss
                best_ndcg = ndcg
                best_hit = hit
                best_rank = rank
                torch.save(model.state_dict(), os.path.join(MODEL_PATH, model_name + '.pt'))

            tbar.set_description(f'Epoch: {epoch:3d}| Train loss: {train_loss:.5f}| NDCG@10: {ndcg:.5f}| HIT@10: {hit:.5f}| Rank@10: {rank:.5f}')
    
print(f'BEST | Epoch: {best_epoch:3d}| Train loss: {best_train_loss:.5f}| NDCG@10: {best_ndcg:.5f}| HIT@10: {best_hit:.5f}| Rank@10: {best_rank:.5f}')

Epoch:   1| Train loss: 9.69904| NDCG@10: 0.01621| HIT@10: 0.02397| Rank@10: 3.20638: 100%|██████████| 1/1 [00:40<00:00, 40.99s/it]
Epoch:   2| Train loss: 8.65676| NDCG@10: 0.06793| HIT@10: 0.09337| Rank@10: 2.76406: 100%|██████████| 1/1 [00:39<00:00, 39.08s/it]
Epoch:   3| Train loss: 7.49686| NDCG@10: 0.10010| HIT@10: 0.13616| Rank@10: 2.67491: 100%|██████████| 1/1 [00:35<00:00, 35.74s/it]
Epoch:   4| Train loss: 6.62092| NDCG@10: 0.11920| HIT@10: 0.16043| Rank@10: 2.57343: 100%|██████████| 1/1 [00:39<00:00, 39.11s/it]
Epoch:   5| Train loss: 5.92827| NDCG@10: 0.13163| HIT@10: 0.17517| Rank@10: 2.51878: 100%|██████████| 1/1 [00:38<00:00, 38.47s/it]
Epoch:   6| Train loss: 5.35128| NDCG@10: 0.13877| HIT@10: 0.18531| Rank@10: 2.54568: 100%|██████████| 1/1 [00:35<00:00, 35.92s/it]
Epoch:   7| Train loss: 4.85633| NDCG@10: 0.14345| HIT@10: 0.19148| Rank@10: 2.53182: 100%|██████████| 1/1 [00:37<00:00, 37.54s/it]
Epoch:   8| Train loss: 4.42212| NDCG@10: 0.14431| HIT@10: 0.19118| Rank@10:

BEST | Epoch:   8| Train loss: 4.42212| NDCG@10: 0.14431| HIT@10: 0.19118| Rank@10: 2.48039


무작위 임베딩을 활용하여 seq 모델 학습

```
헌재 데이터
Epoch:   1| Train loss: 9.69904| NDCG@10: 0.01621| HIT@10: 0.02397| Rank@10: 3.20638: 100%|██████████| 1/1 [00:40<00:00, 40.99s/it]
Epoch:   2| Train loss: 8.65676| NDCG@10: 0.06793| HIT@10: 0.09337| Rank@10: 2.76406: 100%|██████████| 1/1 [00:39<00:00, 39.08s/it]
Epoch:   3| Train loss: 7.49686| NDCG@10: 0.10010| HIT@10: 0.13616| Rank@10: 2.67491: 100%|██████████| 1/1 [00:35<00:00, 35.74s/it]
Epoch:   4| Train loss: 6.62092| NDCG@10: 0.11920| HIT@10: 0.16043| Rank@10: 2.57343: 100%|██████████| 1/1 [00:39<00:00, 39.11s/it]
Epoch:   5| Train loss: 5.92827| NDCG@10: 0.13163| HIT@10: 0.17517| Rank@10: 2.51878: 100%|██████████| 1/1 [00:38<00:00, 38.47s/it]
Epoch:   6| Train loss: 5.35128| NDCG@10: 0.13877| HIT@10: 0.18531| Rank@10: 2.54568: 100%|██████████| 1/1 [00:35<00:00, 35.92s/it]
Epoch:   7| Train loss: 4.85633| NDCG@10: 0.14345| HIT@10: 0.19148| Rank@10: 2.53182: 100%|██████████| 1/1 [00:37<00:00, 37.54s/it]
Epoch:   8| Train loss: 4.42212| NDCG@10: 0.14431| HIT@10: 0.19118| Rank@10: 2.48039: 100%|██████████| 1/1 [00:39<00:00, 39.13s/it]
Epoch:   9| Train loss: 4.03473| NDCG@10: 0.14344| HIT@10: 0.19179| Rank@10: 2.56714: 100%|██████████| 1/1 [00:38<00:00, 38.37s/it]
Epoch:  10| Train loss: 3.69059| NDCG@10: 0.14370| HIT@10: 0.19108| Rank@10: 2.50894: 100%|██████████| 1/1 [00:37<00:00, 37.97s/it]
Epoch:  11| Train loss: 3.37700| NDCG@10: 0.14065| HIT@10: 0.18679| Rank@10: 2.52334: 100%|██████████| 1/1 [00:38<00:00, 38.51s/it]
Epoch:  12| Train loss: 3.09332| NDCG@10: 0.13861| HIT@10: 0.18480| Rank@10: 2.52539: 100%|██████████| 1/1 [00:36<00:00, 36.42s/it]
Epoch:  13| Train loss: 2.83717| NDCG@10: 0.13549| HIT@10: 0.18169| Rank@10: 2.56694: 100%|██████████| 1/1 [00:36<00:00, 36.19s/it]
Epoch:  14| Train loss: 2.60283| NDCG@10: 0.13349| HIT@10: 0.17950| Rank@10: 2.58835: 100%|██████████| 1/1 [00:35<00:00, 35.53s/it]
Epoch:  15| Train loss: 2.38820| NDCG@10: 0.13225| HIT@10: 0.17721| Rank@10: 2.57583: 100%|██████████| 1/1 [00:36<00:00, 36.12s/it]
Epoch:  16| Train loss: 2.18880| NDCG@10: 0.13035| HIT@10: 0.17573| Rank@10: 2.62246: 100%|██████████| 1/1 [00:36<00:00, 36.40s/it]
Epoch:  17| Train loss: 2.00677| NDCG@10: 0.12870| HIT@10: 0.17282| Rank@10: 2.59044: 100%|██████████| 1/1 [00:37<00:00, 37.12s/it]
Epoch:  18| Train loss: 1.84272| NDCG@10: 0.12744| HIT@10: 0.17124| Rank@10: 2.59589: 100%|██████████| 1/1 [00:37<00:00, 37.84s/it]
Epoch:  19| Train loss: 1.69169| NDCG@10: 0.12531| HIT@10: 0.17002| Rank@10: 2.65867: 100%|██████████| 1/1 [00:36<00:00, 36.29s/it]
Epoch:  20| Train loss: 1.54895| NDCG@10: 0.12440| HIT@10: 0.16594| Rank@10: 2.54640: 100%|██████████| 1/1 [00:36<00:00, 36.50s/it]
BEST | Epoch:   8| Train loss: 4.42212| NDCG@10: 0.14431| HIT@10: 0.19118| Rank@10: 2.48039


과거 데이터
Epoch:   1| Train loss: 9.47542| NDCG@10: 0.00798| HIT@10: 0.01276| Rank@10: 3.72727: 100%|██████████| 1/1 [00:28<00:00, 28.94s/it]
Epoch:   2| Train loss: 8.68927| NDCG@10: 0.05917| HIT@10: 0.08264| Rank@10: 2.77081: 100%|██████████| 1/1 [00:29<00:00, 29.60s/it]
Epoch:   3| Train loss: 7.68180| NDCG@10: 0.09417| HIT@10: 0.12863| Rank@10: 2.70974: 100%|██████████| 1/1 [00:24<00:00, 24.00s/it]
Epoch:   4| Train loss: 6.81525| NDCG@10: 0.11760| HIT@10: 0.15925| Rank@10: 2.59709: 100%|██████████| 1/1 [00:26<00:00, 26.39s/it]
Epoch:   5| Train loss: 6.11427| NDCG@10: 0.13399| HIT@10: 0.17942| Rank@10: 2.54545: 100%|██████████| 1/1 [00:27<00:00, 27.16s/it]
Epoch:   6| Train loss: 5.52477| NDCG@10: 0.14506| HIT@10: 0.19457| Rank@10: 2.54748: 100%|██████████| 1/1 [00:26<00:00, 26.68s/it]
Epoch:   7| Train loss: 5.01476| NDCG@10: 0.15132| HIT@10: 0.20176| Rank@10: 2.56169: 100%|██████████| 1/1 [00:27<00:00, 27.37s/it]
Epoch:   8| Train loss: 4.56439| NDCG@10: 0.15281| HIT@10: 0.20346| Rank@10: 2.52204: 100%|██████████| 1/1 [00:27<00:00, 27.79s/it]
Epoch:   9| Train loss: 4.15730| NDCG@10: 0.15472| HIT@10: 0.20555| Rank@10: 2.50056: 100%|██████████| 1/1 [00:27<00:00, 27.91s/it]
Epoch:  10| Train loss: 3.79333| NDCG@10: 0.15704| HIT@10: 0.20787| Rank@10: 2.49498: 100%|██████████| 1/1 [00:28<00:00, 28.53s/it]
Epoch:  11| Train loss: 3.45896| NDCG@10: 0.15463| HIT@10: 0.20439| Rank@10: 2.48071: 100%|██████████| 1/1 [00:24<00:00, 24.63s/it]
Epoch:  12| Train loss: 3.15628| NDCG@10: 0.15253| HIT@10: 0.20431| Rank@10: 2.57283: 100%|██████████| 1/1 [00:26<00:00, 26.88s/it]
Epoch:  13| Train loss: 2.87723| NDCG@10: 0.15219| HIT@10: 0.20393| Rank@10: 2.59629: 100%|██████████| 1/1 [00:27<00:00, 27.29s/it]
Epoch:  14| Train loss: 2.62568| NDCG@10: 0.15028| HIT@10: 0.20153| Rank@10: 2.56157: 100%|██████████| 1/1 [00:26<00:00, 26.95s/it]
Epoch:  15| Train loss: 2.38836| NDCG@10: 0.14826| HIT@10: 0.19821| Rank@10: 2.56357: 100%|██████████| 1/1 [00:27<00:00, 27.04s/it]
Epoch:  16| Train loss: 2.17512| NDCG@10: 0.14657| HIT@10: 0.19542| Rank@10: 2.53283: 100%|██████████| 1/1 [00:26<00:00, 26.84s/it]
Epoch:  17| Train loss: 1.97474| NDCG@10: 0.14398| HIT@10: 0.19264| Rank@10: 2.56541: 100%|██████████| 1/1 [00:27<00:00, 27.45s/it]
Epoch:  18| Train loss: 1.79934| NDCG@10: 0.14256| HIT@10: 0.19117| Rank@10: 2.58674: 100%|██████████| 1/1 [00:26<00:00, 26.67s/it]
Epoch:  19| Train loss: 1.63086| NDCG@10: 0.14038| HIT@10: 0.18955| Rank@10: 2.59502: 100%|██████████| 1/1 [00:28<00:00, 28.03s/it]
Epoch:  20| Train loss: 1.47892| NDCG@10: 0.13974| HIT@10: 0.18808| Rank@10: 2.63831: 100%|██████████| 1/1 [00:26<00:00, 26.31s/it]
BEST | Epoch:  10| Train loss: 3.79333| NDCG@10: 0.15704| HIT@10: 0.20787| Rank@10: 2.49498

```